<a href="https://colab.research.google.com/github/ADGVLOGS/Cape-Town-Project/blob/main/DurbanvilleSuburbs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Raw Data and Importing

In [ ]:
import folium
import re
#locations
tablemountainlatandlong = [-33.903294897303354, 18.420746651123252]

#suburbs
suburbs = ["Amanda Glen (Residential)",
"Aurora (Residential)",
"Avalon Estate (Residential)",
"Bergsig (Residential)",
"Brentwood Park (Residential)",
"Country Places (Residential)",
"D'urbanvale (Residential)",
"Durbanville Hills (Residential)",
"Durbell (Residential)",
"Durmonte (Residential)",
"Everglen (Residential)",
"Eversdal (Residential)",
"Eversdal Heights (Residential)",
"Goedemoed (Residential)",
"Graanendal (Residential)",
"Halali (Residential)",
"Kenridge (Residential)",
"Kenridge Heights (Residential)",
"Klein Nederburg (Residential)",
"Langeberg Village (Residential)",
"Morningstar (Residential)",
"Pinehurst (Residential)",
"Proteaville (Residential)",
"Rosedale (Residential)",
"Schoongezicht (Residential)",
"Sonstraal (Residential)",
"Sonstraal Heights (Residential)",
"Tara (Residential)",
"The Crest (Residential)",
"Uitzicht (Residential)",
"Valmary Park (Residential)",
"Vergesig (Residential)",
"Vierlanden (Residential)",
"Vygeboom (Residential)",
"Welgevonden Estate (Residential)",
"Wellway Park (Residential)",
"Wellway Park East (Residential)"]


# Cleaning our suburbs to remove brackets from list
for index,suburb in enumerate(suburbs):
    suburbs[index] = re.sub("[\(\[].*?[\)\]]", "", suburb)

In [ ]:
len(suburbs)

37

# Features Engineering

#### Adding (Lat and Long) and Distance between Table Mountain to our dataset


<h5>Haversine's Formula to calculate distance between two lat and two long points on the Earth</h5>
<img src = "https://user-images.githubusercontent.com/2789198/27240436-e9a459da-52d4-11e7-8f84-f96d0b312859.png">

In [ ]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371
    return c * r

#### Generating lat and long for our Cape Town Suburbs

In [ ]:
from geopy.geocoders import Nominatim
import time
from pprint import pprint

def ReturnLatAndLongOfSuburb(suburb):
  app = Nominatim(user_agent="adg-csresearch")
  location = app.geocode(suburb + ' , Cape Town').raw
  data = []
  for key,value in location.items():
      if key == 'lat':
        data.append(value)
      if key == 'lon':
        data.append(value)
  return data

In [ ]:
!pip install tqdm

### Performing Calculations

This block calculates the Lat and Long for all Cape Town Suburbs

In [ ]:
from tqdm.notebook import tqdm
lat = []
long = []
newsuburb = []
for index,suburb in enumerate(tqdm(suburbs)):
    try:
        data = ReturnLatAndLongOfSuburb(suburb)
        lat.append(data[0])
        long.append(data[1])
        newsuburb.append(suburb)
    except Exception as e:
        print('Error:','removing {0} suburb from dataset as lat and long cannot be processed'.format(suburb))

  0%|          | 0/37 [00:00<?, ?it/s]

Error: removing Country Places  suburb from dataset as lat and long cannot be processed
Error: removing D'urbanvale  suburb from dataset as lat and long cannot be processed
Error: removing Eversdal Heights  suburb from dataset as lat and long cannot be processed
Error: removing Halali  suburb from dataset as lat and long cannot be processed
Error: removing Klein Nederburg  suburb from dataset as lat and long cannot be processed
Error: removing Sonstraal Heights  suburb from dataset as lat and long cannot be processed
Error: removing Wellway Park  suburb from dataset as lat and long cannot be processed
Error: removing Wellway Park East  suburb from dataset as lat and long cannot be processed


Working out distances from Table Mountain to the Suburbs

In [ ]:
distance = []

for index,suburb in enumerate(tqdm(newsuburb)):
    distance.append(haversine(-33.903294897303354, 18.420746651123252,float(lat[index]),float(long[index])))

  0%|          | 0/29 [00:00<?, ?it/s]

### Our final dataset we have compiled for this report

# Calculating Elevation for Suburbs



[API Documentation - Open Elevation](https://github.com/Jorl17/open-elevation/blob/master/docs/api.md)

In [ ]:
import requests
def ReturnElevation(lat,long):
  r = requests.get('https://api.open-elevation.com/api/v1/lookup?locations={0},{1}'.format(lat,long))
  data = r.json()
  for key,info in data.items():
      if key == 'results':
          return info[0].get('elevation')

In [ ]:
elevation = []

for index,suburb in enumerate(tqdm(newsuburb)):
    elevation.append(ReturnElevation(float(lat[index]),float(long[index])))

  0%|          | 0/29 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame(list(zip(newsuburb, lat,long,distance,elevation)),
               columns =['Suburb', 'Latitude','Longtitude','Distance(KM)','Height above sea level(m)'])
df = df.drop_duplicates(subset=['Suburb'])
df = df.sort_values(by='Distance(KM)',ascending=True)
df = df.sort_values(by='Height above sea level(m)',ascending=False)
df.to_csv('Durbanville.csv')
df

,Suburb,Latitude,Longtitude,Distance(KM),Height above sea level(m)
5,Durbanville Hills,-33.8480556,18.6386111,24.915635,237
25,Vergesig,-33.8394693,18.6310927,24.338205,235
1,Aurora,-33.8302778,18.6319444,24.713736,201
8,Everglen,-33.8536048,18.6475955,25.762696,179
6,Durbell,-33.8586778,18.6129234,21.880779,176
12,Kenridge,-33.8580556,18.6372222,24.538978,173
23,Uitzicht,-33.8815205,18.6182608,22.082240,164
9,Eversdal,-33.8616667,18.6486111,25.714655,161
13,Kenridge Heights,38.5867699,-90.3334527,12066.959988,157
19,Schoongezicht,-33.8169444,18.6475,26.806990,157


# Now the fun part visualizing the dataset

Generating our map

In [ ]:
m = folium.Map(location=tablemountainlatandlong)
folium.Circle([-33.967375918362606, 18.416404913543296],
                    radius=1250
                   ).add_to(m)

Drawing circles around suburbs

In [ ]:
m = folium.Map(location=tablemountainlatandlong)

m.add_child(folium.Circle([-33.903294897303354, 18.420746651123252],
                    radius=1250
                   ).add_to(m))
for index, row in df.iterrows():
    text = 'Suburb: {0}\n\n\n\n\n\nDistance from  V&A Waterfront: {1}\n\n\n\n\n\n\n\n\n\nHeight above sea level(m): {2}'.format(str(row['Suburb']),str(row['Distance(KM)'])+'KM',row['Height above sea level(m)'])
    m.add_child(folium.Marker([float(row['Latitude']),float(row['Longtitude'])],popup=text,tooltip='Click for more information!'))

In [ ]:
m

Saving map to .html

In [ ]:
try:
  m.save(outfile= "cptsuburbdata.html")
  print('map saved to .html')
except Exception as e:
  print('Error file couldnt save: ',e)

map saved to .html


Notebook compiled by Ashlin Darius Govindasamy 2021